# Laporan Praktikum 2 – Kecerdasan Komputasional  
## Proof by Resolution, Forward & Backward Chaining, dan First-Order Logic  


### Identitas Mahasiswa
- Nama  : Wayan Raditya Putra  
- NRP   : 5054241029  
- Program Studi : Rekayasa Kecerdasan Artifisial
- Angkatan : 2024  


### Tujuan Praktikum
Praktikum ini bertujuan untuk:  
1. Memahami konsep dasar Proof by Resolution sebagai teknik inferensi dalam logika.  
2. Mempelajari mekanisme Forward Chaining dan Backward Chaining untuk penalaran berbasis aturan.  
3. Mengenal representasi First-Order Logic (FOL) dan bagaimana FOL digunakan dalam basis pengetahuan.  
4. Mengimplementasikan berbagai teknik inferensi untuk menyelesaikan permasalahan berbasis logika komputasional.  


### Deskripsi Singkat
Pada praktikum kedua ini, mahasiswa diminta untuk:  
- Menyusun representasi basis pengetahuan dalam bentuk proposisi maupun logika orde pertama.  
- Menerapkan teknik inferensi seperti resolution, forward chaining, backward chaining.  
- Melakukan percobaan terhadap kasus sederhana untuk melihat bagaimana sistem inferensi menghasilkan kesimpulan dari basis pengetahuan.  


## Instalasi dan Import Library

**Tujuan:**  
Menyiapkan library dan modul yang diperlukan sebelum menjalankan praktikum. Pada tahap ini dilakukan instalasi paket eksternal serta import fungsi-fungsi yang sudah tersedia di file pendukung.

```python
%pip install ipythonblocks
%pip install qpsolvers

from utils import *
from logic import *
from notebook import psource
````
**Penjelasan:**

* `%pip install ipythonblocks` → menginstal library **ipythonblocks** yang digunakan untuk menampilkan blok warna pada notebook.
* `%pip install qpsolvers` → menginstal library **qpsolvers** yang digunakan untuk menyelesaikan masalah Quadratic Programming.
* `from utils import *` → mengimpor seluruh fungsi dari file `utils.py` yang berisi kumpulan fungsi bantu.
* `from logic import *` → mengimpor seluruh fungsi dari file `logic.py` yang berisi implementasi logika proposisional dan inference.
* `from notebook import psource` → mengimpor fungsi `psource` dari `notebook.py`, yang digunakan untuk menampilkan source code suatu fungsi langsung di notebook.



In [150]:
%pip install ipythonblocks
%pip install qpsolvers
from utils import *
from logic import *
from notebook import psource
import inspect

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Inferensi pada Basis Pengetahuan Proposisional

**Tujuan:**  
Pada bagian ini kita akan mempelajari dua teknik utama yang digunakan untuk memeriksa apakah suatu kalimat (sentence) dapat di-*entail* oleh sebuah basis pengetahuan (`KB`).  






### Proof by Resolution

Tujuan kita adalah memeriksa apakah $KB \vDash \alpha$, yaitu apakah $KB \implies \alpha$ bernilai benar di setiap model.  

Contoh: kita ingin memeriksa apakah $P \implies Q$ valid.  
Untuk itu kita periksa keterpuasan dari $\neg (P \implies Q)$, yang bisa ditulis ulang menjadi:  

$$
P \land \neg Q
$$

Jika ekspresi $P \land \neg Q$ **tidak terpuaskan (unsatisfiable)**, maka $P \implies Q$ pasti benar di semua model.  


### Konsep Dasar

Teknik ini dikenal sebagai **pembuktian dengan kontradiksi** (*proof by contradiction*).  
Langkahnya: kita mengasumsikan bahwa $\alpha$ salah, lalu menunjukkan bahwa asumsi ini menimbulkan kontradiksi dengan fakta-fakta dalam $KB$.  


### Aturan Resolusi

Aturan resolusi berbunyi:  

$$
(l_1 \lor \dots \lor l_k), \quad (m_1 \lor \dots \lor m_n), \quad (l_i \equiv \neg m_j) 
\;\;\;\Rightarrow\;\;\;
(l_1 \lor \dots \lor l_{i-1} \lor l_{i+1} \lor \dots \lor l_k \lor m_1 \lor \dots \lor m_{j-1} \lor m_{j+1} \lor \dots \lor m_n)
$$


### Hasil Resolusi

Proses resolusi dijalankan berulang sampai salah satu kondisi berikut:

1. Tidak ada klausa baru yang bisa ditambahkan → artinya $KB \nvDash \alpha$.  
2. Ditemukan **klausa kosong** → artinya $KB \vDash \alpha$.  

Klausa kosong setara dengan **False**, karena hanya bisa muncul dari resolusi dua klausa saling berlawanan, misalnya $P$ dan $\neg P$.


# Konversi Kalimat ke Bentuk Normal Konjungtif (CNF)

Dalam pembuktian dengan **resolusi**, algoritma tidak bisa langsung menangani kalimat logika yang rumit.  
Karena itu, kalimat yang mengandung **implikasi (→)** dan **bi-implikasi (↔)** harus disederhanakan dulu.

👉 Fakta penting: **setiap kalimat logika proposisional setara dengan bentuk konjungsi dari klausa.**  
Bentuk ini disebut **Conjunctive Normal Form (CNF)**, yaitu **konjungsi (AND)** dari beberapa **disjungsi (OR) literal**.

Contoh CNF:

$$
(A \lor B) \land (\neg B \lor C \lor \neg D) \land (D \lor \neg E)
$$

Bentuk ini sama seperti **Product of Sums (POS)** di elektronika digital.


## Tahapan Konversi ke CNF

### 1. Ubah Bi-Implikasi ke Implikasi
$$
\alpha \iff \beta \equiv (\alpha \implies \beta) \land (\beta \implies \alpha)
$$

Jika bagian kanan berupa kalimat majemuk:
$$
\alpha \iff (\beta \lor \gamma) \equiv (\alpha \implies (\beta \lor \gamma)) \land ((\beta \lor \gamma) \implies \alpha)
$$


### 2. Ubah Implikasi ke Bentuk Setara
$$
\alpha \implies \beta \equiv \neg \alpha \lor \beta
$$


### 3. Pindahkan Negasi ke Literal
Negasi hanya boleh menempel pada proposisi atomik.  
Gunakan **Hukum De Morgan**:

$$
\neg(\alpha \land \beta) \equiv (\neg \alpha \lor \neg \beta)
$$

$$
\neg(\alpha \lor \beta) \equiv (\neg \alpha \land \neg \beta)
$$


### 4. Distribusikan Disjungsi terhadap Konjungsi
Agar sesuai format CNF, lakukan distribusi:

$$
(\alpha \lor (\beta \land \gamma)) \equiv (\alpha \lor \beta) \land (\alpha \lor \gamma)
$$

Bentuk akhir yang kita inginkan:

$$
(\alpha_1 \lor \alpha_2 \lor \dots) \land (\beta_1 \lor \beta_2 \lor \dots) \land (\gamma_1 \lor \gamma_2 \lor \dots)
$$


## Inti Proses
- CNF = **“AND of ORs”** (konjungsi dari disjungsi literal).  
- Proses konversi = hilangkan ↔ dan →, dorong negasi ke literal, lalu distribusikan OR atas AND.  
- Dengan CNF, kalimat siap dipakai dalam algoritma **resolusi**.




In [151]:
print(inspect.getsource(to_cnf))

def to_cnf(s):
    """
    [Page 253]
    Convert a propositional logical sentence to conjunctive normal form.
    That is, to the form ((A | ~B | ...) & (B | C | ...) & ...)
    >>> to_cnf('~(B | C)')
    (~B & ~C)
    """
    s = expr(s)
    if isinstance(s, str):
        s = expr(s)
    s = eliminate_implications(s)  # Steps 1, 2 from p. 253
    s = move_not_inwards(s)  # Step 3
    return distribute_and_over_or(s)  # Step 4



## Fungsi `to_cnf`
Untuk mengubah sebuah kalimat logika proposisional ke bentuk **Conjunctive Normal Form (CNF)**, kita gunakan fungsi `to_cnf`.  

```python
def to_cnf(s): 
    """
    [Page 253]
    Convert a propositional logical sentence to conjunctive normal form.
    That is, to the form ((A | ~B | ...) & (B | C | ...) & ...)
    >>> to_cnf('~(B | C)')
    (~B & ~C)
    """
    s = expr(s)
    if isinstance(s, str):
        s = expr(s)
    s = eliminate_implications(s)  # Steps 1, 2 from p. 253
    s = move_not_inwards(s)        # Step 3
    return distribute_and_over_or(s)  # Step 4
```

### Penjelasan Tahapan

1. **`eliminate_implications(s)`**

   * Menyelesaikan **Langkah 1 dan 2** dari proses konversi.
   * Semua bi-implikasi (\$\alpha \iff \beta\$) diubah menjadi dua implikasi.
   * Semua implikasi (\$\alpha \implies \beta\$) diubah menjadi bentuk setara (\$\neg \alpha \lor \beta\$).

2. **`move_not_inwards(s)`**

   * Menyelesaikan **Langkah 3**.
   * Negasi digeser ke dalam agar hanya menempel pada literal, dengan bantuan hukum De Morgan.
   * Contoh:

     $$
     \neg (A \lor B) \equiv (\neg A \land \neg B)
     $$

3. **`distribute_and_over_or(s)`**

   * Menyelesaikan **Langkah 4**.
   * Melakukan distribusi disjungsi terhadap konjungsi, sehingga hasil akhir berbentuk **konjungsi dari disjungsi literal**.
   * Contoh:

     $$
     (A \lor (B \land C)) \equiv (A \lor B) \land (A \lor C)
     $$


### Intinya

* Fungsi `to_cnf` = implementasi praktis dari 4 langkah konversi CNF.
* Setiap kalimat proposisional bisa dipaksa ke CNF → siap dipakai untuk **algoritma resolusi**.



In [152]:
print(inspect.getsource(eliminate_implications))
print(inspect.getsource(move_not_inwards))
print(inspect.getsource(distribute_and_over_or))


def eliminate_implications(s):
    """Change implications into equivalent form with only &, |, and ~ as logical operators."""
    s = expr(s)
    if not s.args or is_symbol(s.op):
        return s  # Atoms are unchanged.
    args = list(map(eliminate_implications, s.args))
    a, b = args[0], args[-1]
    if s.op == '==>':
        return b | ~a
    elif s.op == '<==':
        return a | ~b
    elif s.op == '<=>':
        return (a | ~b) & (b | ~a)
    elif s.op == '^':
        assert len(args) == 2  # TODO: relax this restriction
        return (a & ~b) | (~a & b)
    else:
        assert s.op in ('&', '|', '~')
        return Expr(s.op, *args)

def move_not_inwards(s):
    """Rewrite sentence s by moving negation sign inward.
    >>> move_not_inwards(~(A | B))
    (~A & ~B)
    """
    s = expr(s)
    if s.op == '~':
        def NOT(b):
            return move_not_inwards(~b)

        a = s.args[0]
        if a.op == '~':
            return move_not_inwards(a.args[0])  # ~~A ==> A
   


## Fungsi Pendukung `to_cnf`

Agar `to_cnf` bekerja, ada tiga fungsi utama yang menjalankan proses sesuai 4 langkah konversi CNF:

### 1. `eliminate_implications(s)`

```python
def eliminate_implications(s):
    """Change implications into equivalent form with only &, |, and ~ as logical operators."""
    s = expr(s)
    if not s.args or is_symbol(s.op):
        return s  # Atoms are unchanged.
    args = list(map(eliminate_implications, s.args))
    a, b = args[0], args[-1]
    if s.op == '==>':
        return b | ~a
    elif s.op == '<==':
        return a | ~b
    elif s.op == '<=>':
        return (a | ~b) & (b | ~a)
    elif s.op == '^':
        assert len(args) == 2
        return (a & ~b) | (~a & b)
    else:
        assert s.op in ('&', '|', '~')
        return Expr(s.op, *args)
```

**Fungsi:**

* Menghilangkan semua **implikasi (⇒)** dan **bi-implikasi (⇔)**.
* Hasil akhirnya hanya mengandung operator `&` (AND), `|` (OR), dan `~` (NOT).
* Contoh:

$$
(P \implies Q) \equiv (\neg P \lor Q)
$$

$$
(P \iff Q) \equiv (P \implies Q) \land (Q \implies P)
$$


### 2. `move_not_inwards(s)`

```python
def move_not_inwards(s):
    """Rewrite sentence s by moving negation sign inward.
    >>> move_not_inwards(~(A | B))
    (~A & ~B)
    """
    s = expr(s)
    if s.op == '~':
        def NOT(b):
            return move_not_inwards(~b)

        a = s.args[0]
        if a.op == '~':
            return move_not_inwards(a.args[0])  # ~~A ==> A
        if a.op == '&':
            return associate('|', list(map(NOT, a.args)))
        if a.op == '|':
            return associate('&', list(map(NOT, a.args)))
        return s
    elif is_symbol(s.op) or not s.args:
        return s
    else:
        return Expr(s.op, *list(map(move_not_inwards, s.args)))
```

**Fungsi:**

* Memindahkan **negasi** supaya hanya menempel pada literal (variabel atomik).
* Menggunakan **Hukum De Morgan** dan penyederhanaan ganda:

$$
\neg (A \lor B) \equiv (\neg A \land \neg B)
$$

$$
\neg (A \land B) \equiv (\neg A \lor \neg B)
$$

$$
\neg(\neg A) \equiv A
$$


### 3. `distribute_and_over_or(s)`

```python
def distribute_and_over_or(s):
    """Given a sentence s consisting of conjunctions and disjunctions
    of literals, return an equivalent sentence in CNF.
    >>> distribute_and_over_or((A & B) | C)
    ((A | C) & (B | C))
    """
    s = expr(s)
    if s.op == '|':
        s = associate('|', s.args)
        if s.op != '|':
            return distribute_and_over_or(s)
        if len(s.args) == 0:
            return False
        if len(s.args) == 1:
            return distribute_and_over_or(s.args[0])
        conj = first(arg for arg in s.args if arg.op == '&')
        if not conj:
            return s
        others = [a for a in s.args if a is not conj]
        rest = associate('|', others)
        return associate('&', [distribute_and_over_or(c | rest)
                               for c in conj.args])
    elif s.op == '&':
        return associate('&', list(map(distribute_and_over_or, s.args)))
    else:
        return s
```

**Fungsi:**

* Melakukan distribusi **OR terhadap AND**, supaya hasil akhir sesuai format CNF.
* Contoh:

$$
(A \land B) \lor C \equiv (A \lor C) \land (B \lor C)
$$


## Alur Lengkap

Jadi, `to_cnf` menjalankan ketiga fungsi ini berurutan:

1. **Hilangkan implikasi** → `eliminate_implications(s)`
2. **Pindahkan negasi** → `move_not_inwards(s)`
3. **Distribusikan OR atas AND** → `distribute_and_over_or(s)`

Hasil akhirnya selalu berbentuk:

$$
(\alpha_1 \lor \alpha_2 \lor \dots) \land (\beta_1 \lor \beta_2 \lor \dots) \land \dots
$$

siap dipakai dalam algoritma **resolusi**.




Mari kita coba untuk konversi beberapa kalimat menjadi cnf

In [153]:
A, B, C, D = expr('A, B, C, D')
to_cnf(A |'<=>'| B)

((A | ~B) & (B | ~A))

In [154]:
to_cnf(A |'<=>'| (B & C))

((A | ~B | ~C) & (B | ~A) & (C | ~A))

In [155]:
to_cnf(A & (B | (C & D)))

(A & (C | B) & (D | B))

In [156]:
to_cnf((A |'<=>'| ~B) |'==>'| (C | ~D))

((B | ~A | C | ~D) & (A | ~A | C | ~D) & (B | ~B | C | ~D) & (A | ~B | C | ~D))

## Implementasi `pl_resolution`

Setelah memahami fungsi `to_cnf`, kita bisa lihat bagaimana fungsi ini digunakan dalam algoritma **Proof by Resolution**.  
Berikut kode dari fungsi `pl_resolution`:

```python
def pl_resolution(kb, alpha):
    """
    [Figure 7.12]
    Propositional-logic resolution: say if alpha follows from KB.
    >>> pl_resolution(horn_clauses_KB, A)
    True
    """
    clauses = kb.clauses + conjuncts(to_cnf(~alpha))
    new = set()
    while True:
        n = len(clauses)
        pairs = [(clauses[i], clauses[j])
                 for i in range(n) for j in range(i + 1, n)]
        for (ci, cj) in pairs:
            resolvents = pl_resolve(ci, cj)
            if False in resolvents:
                return True
            new = new.union(set(resolvents))
        if new.issubset(set(clauses)):
            return False
        for c in new:
            if c not in clauses:
                clauses.append(c)
```

### Alur Kerja `pl_resolution`

1. **Persiapan klausa**

   * Semua klausa dalam basis pengetahuan (`kb.clauses`) digabung dengan hasil konversi CNF dari negasi query `~alpha`.
   * Hal ini sesuai prinsip resolusi: untuk membuktikan $\text{KB} \vDash \alpha$, kita cek apakah $\text{KB} \land \neg \alpha$ tidak terpuaskan.

2. **Inisialisasi**

   * Variabel `new` dibuat sebagai himpunan kosong untuk menampung resolvent (klausa hasil resolusi).

3. **Loop utama**

   * Semua pasangan klausa dalam `clauses` diambil.
   * Untuk setiap pasangan `(ci, cj)`, dijalankan fungsi `pl_resolve` untuk mencari resolvent.

4. **Pengecekan kontradiksi**

   * Jika ada resolvent yang menghasilkan `False` (klausa kosong), berarti kontradiksi ditemukan → $\text{KB} \vDash \alpha$.
   * Fungsi langsung mengembalikan `True`.

5. **Update klausa**

   * Semua resolvent baru dimasukkan ke dalam `new`.
   * Jika `new` hanyalah subset dari klausa lama (tidak ada klausa baru yang muncul), artinya tidak ada resolusi lebih lanjut → algoritma berhenti dengan hasil `False` (tidak terbukti).
   * Jika ada klausa baru, tambahkan ke `clauses` lalu ulangi proses.

### Intuisi

* **Kunci resolusi:** cari pasangan klausa yang punya literal saling berlawanan, lalu gabungkan sisanya.
* **Tujuan:** terus menghasilkan klausa baru sampai:

  1. Ketemu **klausa kosong** (kontradiksi → query terbukti).
  2. Tidak ada klausa baru lagi (tidak terbukti).

Dengan demikian, `pl_resolution` bekerja sebagai prosedur sistematis untuk mengecek entailment menggunakan prinsip **proof by contradiction** dengan bantuan konversi ke **CNF**.


In [157]:
print(inspect.getsource(pl_resolution))


def pl_resolution(kb, alpha):
    """
    [Figure 7.12]
    Propositional-logic resolution: say if alpha follows from KB.
    >>> pl_resolution(horn_clauses_KB, A)
    True
    """
    clauses = kb.clauses + conjuncts(to_cnf(~alpha))
    new = set()
    while True:
        n = len(clauses)
        pairs = [(clauses[i], clauses[j])
                 for i in range(n) for j in range(i + 1, n)]
        for (ci, cj) in pairs:
            resolvents = pl_resolve(ci, cj)
            if False in resolvents:
                return True
            new = new.union(set(resolvents))
        if new.issubset(set(clauses)):
            return False
        for c in new:
            if c not in clauses:
                clauses.append(c)



In [158]:
pl_resolution(wumpus_kb, ~P11), pl_resolution(wumpus_kb, P11)

(True, False)

In [159]:
pl_resolution(wumpus_kb, ~P22), pl_resolution(wumpus_kb, P22)


(False, False)

## Hasil Uji `pl_resolution` pada Wumpus KB

Kita menguji dua query berbeda pada basis pengetahuan Wumpus menggunakan fungsi `pl_resolution`.

### 1. Query pada Kotak (1,1)

```python
pl_resolution(wumpus_kb, ~P11), pl_resolution(wumpus_kb, P11)
# Hasil: (True, False)
```

**Penjelasan:**
* `pl_resolution(wumpus_kb, ~P11) → True`
  Artinya basis pengetahuan **dapat membuktikan bahwa di kotak (1,1) tidak ada pit**.
  Hal ini sesuai aturan awal permainan Wumpus World: posisi awal pemain (1,1) selalu aman.

* `pl_resolution(wumpus_kb, P11) → False`
  Basis pengetahuan **tidak bisa membuktikan bahwa ada pit di (1,1)**, karena memang informasi dalam KB justru menyatakan sebaliknya.


### 2. Query pada Kotak (2,2)

```python
pl_resolution(wumpus_kb, ~P22), pl_resolution(wumpus_kb, P22)
# Hasil: (False, False)
```

**Penjelasan:**

* `pl_resolution(wumpus_kb, ~P22) → False`
  Basis pengetahuan **tidak bisa membuktikan bahwa (2,2) bebas pit**.
  Informasi yang ada belum cukup untuk menyimpulkan aman atau tidak.

* `pl_resolution(wumpus_kb, P22) → False`
  Basis pengetahuan **juga tidak bisa membuktikan bahwa ada pit di (2,2)**.

Dengan kata lain, posisi (2,2) masih **tidak diketahui statusnya** berdasarkan informasi yang tersedia.

### Kesimpulan

* Pada kotak awal (1,1), KB dengan jelas **meng-entail** bahwa tidak ada pit.
* Pada kotak (2,2), KB belum punya informasi yang cukup, sehingga baik `P22` maupun `~P22` tidak bisa dibuktikan.



### Forward dan Backward Chaining

Sebelumnya kita sudah membahas dua algoritma untuk mengecek apakah sebuah kalimat di-*entail* oleh basis pengetahuan (`KB`).  
Di sini kita perkenalkan **algoritma ketiga**, yaitu *forward chaining* dan *backward chaining*.  

Perbedaan utamanya: tujuan sekarang adalah menentukan apakah basis pengetahuan yang hanya berisi **definite clause** dapat meng-entail sebuah simbol proposisi tunggal $q$ (query).  
Namun, ada satu syarat penting: **basis pengetahuan hanya boleh berisi Horn Clauses**.

#### Horn Clauses

- **Horn Clause** adalah disjungsi (OR) dari beberapa literal dengan **paling banyak satu literal positif**.  
- Jika sebuah Horn Clause memiliki tepat satu literal positif, maka ia disebut **Definite Clause**.  
- Jika Horn Clause hanya berisi literal positif tunggal, ia disebut **Fact**.  

Contoh Horn Clause:  

$$
\neg a \lor \neg b \lor \neg c \lor z
$$

Dengan menggunakan Hukum De Morgan, bentuk ini bisa ditulis ulang menjadi:  

$$
a \land b \land c \;\;\implies\;\; z
$$

Artinya: jika $a$, $b$, dan $c$ semuanya benar, maka $z$ juga dapat disimpulkan benar.  
Bentuk ini mirip dengan cara manusia memproses fakta dan pengetahuan untuk menghasilkan kesimpulan baru.

#### Keunggulan Horn Clauses

1. **Definite Clause dapat ditulis sebagai implikasi**  
   - Premis (body) berupa konjungsi literal positif.  
   - Kesimpulan (head) berupa satu literal positif.  
   - Contoh:  

     $$
     a \land b \implies z
     $$  

   - Sebuah literal positif tunggal disebut **fact**.  
   - Dengan bentuk seperti ini, aturan lebih mudah dipahami maupun diimplementasikan.

2. **Bisa digunakan pada Forward dan Backward Chaining**  
   - *Forward chaining* bekerja dengan menambahkan fakta baru secara progresif.  
   - *Backward chaining* bekerja dengan menelusuri mundur dari query menuju fakta dasar.  
   - Kedua algoritma ini berjalan efisien pada Horn Clauses karena bentuknya sudah sangat terstruktur.

3. **Kompleksitas Linear**  
   - Keputusan entailment dengan Horn Clauses hanya memerlukan traversal setiap klausa dalam basis pengetahuan **paling banyak sekali**.  
   - Hal ini membuat proses inferensi jauh lebih efisien dibanding metode enumerasi atau resolusi umum.


#### Implementasi di Kode

Inferensi dengan *forward chaining* dapat diuji menggunakan fungsi:

```python
pl_fc_entails(KB, q)
```

Catatan penting:

* Basis pengetahuan yang dipakai bukanlah instansi `KB` biasa.
* `KB` di sini adalah objek dari kelas **`PropDefiniteKB`**, turunan dari `PropKB` yang dimodifikasi khusus untuk menyimpan **Definite Clause**.
* Perbedaan utama ada pada metode bantu tambahan yang dapat mengembalikan daftar klausa dalam KB yang memiliki simbol tertentu di bagian premis.

Dengan cara ini, forward chaining dapat lebih cepat menemukan aturan mana saja yang relevan ketika sebuah fakta baru muncul.



In [160]:
print(inspect.getsource(PropDefiniteKB.clauses_with_premise))


    def clauses_with_premise(self, p):
        """Return a list of the clauses in KB that have p in their premise.
        This could be cached away for O(1) speed, but we'll recompute it."""
        return [c for c in self.clauses if c.op == '==>' and p in conjuncts(c.args[0])]




## Fungsi `clauses_with_premise`

```python
psource(PropDefiniteKB.clauses_with_premise)
def clauses_with_premise(self, p):
    """Return a list of the clauses in KB that have p in their premise.
    This could be cached away for O(1) speed, but we'll recompute it."""
    return [c for c in self.clauses if c.op == '==>' and p in conjuncts(c.args[0])]
```

## Fungsi

Digunakan untuk mengambil semua klausa dalam Knowledge Base (KB) yang memiliki simbol tertentu `p` pada bagian premis (bagian kiri implikasi).

## Alur Kerja

1. `self.clauses` → daftar semua klausa dalam basis pengetahuan.
2. `c.op == '==>'` → hanya ambil klausa yang berupa implikasi (definite clause).
3. `c.args[0]` → mengambil bagian premis dari klausa (bagian kiri sebelum `==>`).
4. `conjuncts(c.args[0])` → memecah premis menjadi daftar konjungsi.
5. `p in conjuncts(c.args[0])` → mengecek apakah simbol `p` ada di dalam premis.
6. Hasil akhir berupa list semua klausa yang memenuhi kondisi di atas.

## Intuisi

Premis adalah syarat yang harus dipenuhi agar klausa berlaku.
Fungsi ini digunakan untuk mencari aturan yang relevan dengan simbol tertentu di bagian premis.

## Contoh

Misalkan KB berisi klausa:

1. `A & B ==> C`
2. `D ==> E`
3. `B ==> F`

Jika dipanggil:

```python
clauses_with_premise("B")
```

Maka hasilnya adalah:

* `A & B ==> C`
* `B ==> F`

Kedua klausa tersebut dipilih karena premisnya mengandung `B`.

Let's now have a look at the `pl_fc_entails` algorithm.

In [161]:
print(inspect.getsource(pl_fc_entails))


def pl_fc_entails(kb, q):
    """
    [Figure 7.15]
    Use forward chaining to see if a PropDefiniteKB entails symbol q.
    >>> pl_fc_entails(horn_clauses_KB, expr('Q'))
    True
    """
    count = {c: len(conjuncts(c.args[0])) for c in kb.clauses if c.op == '==>'}
    inferred = defaultdict(bool)
    agenda = [s for s in kb.clauses if is_prop_symbol(s.op)]
    while agenda:
        p = agenda.pop()
        if p == q:
            return True
        if not inferred[p]:
            inferred[p] = True
            for c in kb.clauses_with_premise(p):
                count[c] -= 1
                if count[c] == 0:
                    agenda.append(c.args[1])
    return False



The function accepts a knowledge base `KB` (an instance of `PropDefiniteKB`) and a query `q` as inputs.
<br>
<br>
`count` initially stores the number of symbols in the premise of each sentence in the knowledge base.
<br>
The `conjuncts` helper function separates a given sentence at conjunctions.
<br>
`inferred` is initialized as a *boolean* defaultdict. 
This will be used later to check if we have inferred all premises of each clause of the agenda.
<br>
`agenda` initially stores a list of clauses that the knowledge base knows to be true.
The `is_prop_symbol` helper function checks if the given symbol is a valid propositional logic symbol.
<br>
<br>
We now iterate through `agenda`, popping a symbol `p` on each iteration.
If the query `q` is the same as `p`, we know that entailment holds.
<br>
The agenda is processed, reducing `count` by one for each implication with a premise `p`.
A conclusion is added to the agenda when `count` reaches zero. This means we know all the premises of that particular implication to be true.
<br>
`clauses_with_premise` is a helpful method of the `PropKB` class.
It returns a list of clauses in the knowledge base that have `p` in their premise.
<br>
<br>
Now that we have an idea of how this function works, let's see a few examples of its usage, but we first need to define our knowledge base. We assume we know the following clauses to be true.

In [162]:
clauses = ['(B & F)==>E', 
           '(A & E & F)==>G', 
           '(B & C)==>F', 
           '(A & B)==>D', 
           '(E & F)==>H', 
           '(H & I)==>J',
           'A', 
           'B', 
           'C']

We will now `tell` this information to our knowledge base.

In [163]:
definite_clauses_KB = PropDefiniteKB()
for clause in clauses:
    definite_clauses_KB.tell(expr(clause))

We can now check if our knowledge base entails the following queries.

In [164]:
pl_fc_entails(definite_clauses_KB, expr('G'))

True

In [165]:
pl_fc_entails(definite_clauses_KB, expr('H'))

True

In [166]:
pl_fc_entails(definite_clauses_KB, expr('I'))

False

In [167]:
pl_fc_entails(definite_clauses_KB, expr('J'))

False

## First-Order Logic Knowledge Bases: `FolKB`

The class `FolKB` can be used to represent a knowledge base of First-order logic sentences. You would initialize and use it the same way as you would for `PropKB` except that the clauses are first-order definite clauses. We will see how to write such clauses to create a database and query them in the following sections.

## Criminal KB
In this section we create a `FolKB` based on the following paragraph.<br/>
<em>The law says that it is a crime for an American to sell weapons to hostile nations. The country Nono, an enemy of America, has some missiles, and all of its missiles were sold to it by Colonel West, who is American.</em><br/>
The first step is to extract the facts and convert them into first-order definite clauses. Extracting the facts from data alone is a challenging task. Fortunately, we have a small paragraph and can do extraction and conversion manually. We'll store the clauses in list aptly named `clauses`.

In [168]:
clauses = []

<em>“... it is a crime for an American to sell weapons to hostile nations”</em><br/>
The keywords to look for here are 'crime', 'American', 'sell', 'weapon' and 'hostile'. We use predicate symbols to make meaning of them.

* `Criminal(x)`: `x` is a criminal
* `American(x)`: `x` is an American
* `Sells(x ,y, z)`: `x` sells `y` to `z`
* `Weapon(x)`: `x` is a weapon
* `Hostile(x)`: `x` is a hostile nation

Let us now combine them with appropriate variable naming to depict the meaning of the sentence. The criminal `x` is also the American `x` who sells weapon `y` to `z`, which is a hostile nation.

$\text{American}(x) \land \text{Weapon}(y) \land \text{Sells}(x, y, z) \land \text{Hostile}(z) \implies \text{Criminal} (x)$

In [169]:
clauses.append(expr("(American(x) & Weapon(y) & Sells(x, y, z) & Hostile(z)) ==> Criminal(x)"))

<em>"The country Nono, an enemy of America"</em><br/>
We now know that Nono is an enemy of America. We represent these nations using the constant symbols `Nono` and `America`. the enemy relation is show using the predicate symbol `Enemy`.

$\text{Enemy}(\text{Nono}, \text{America})$

In [170]:
clauses.append(expr("Enemy(Nono, America)"))

<em>"Nono ... has some missiles"</em><br/>
This states the existence of some missile which is owned by Nono. $\exists x \text{Owns}(\text{Nono}, x) \land \text{Missile}(x)$. We invoke existential instantiation to introduce a new constant `M1` which is the missile owned by Nono.

$\text{Owns}(\text{Nono}, \text{M1}), \text{Missile}(\text{M1})$

In [171]:
clauses.append(expr("Owns(Nono, M1)"))
clauses.append(expr("Missile(M1)"))

<em>"All of its missiles were sold to it by Colonel West"</em><br/>
If Nono owns something and it classifies as a missile, then it was sold to Nono by West.

$\text{Missile}(x) \land \text{Owns}(\text{Nono}, x) \implies \text{Sells}(\text{West}, x, \text{Nono})$

In [172]:
clauses.append(expr("(Missile(x) & Owns(Nono, x)) ==> Sells(West, x, Nono)"))

<em>"West, who is American"</em><br/>
West is an American.

$\text{American}(\text{West})$

In [173]:
clauses.append(expr("American(West)"))

We also know, from our understanding of language, that missiles are weapons and that an enemy of America counts as “hostile”.

$\text{Missile}(x) \implies \text{Weapon}(x), \text{Enemy}(x, \text{America}) \implies \text{Hostile}(x)$

In [174]:
clauses.append(expr("Missile(x) ==> Weapon(x)"))
clauses.append(expr("Enemy(x, America) ==> Hostile(x)"))

Now that we have converted the information into first-order definite clauses we can create our first-order logic knowledge base.

In [175]:
crime_kb = FolKB(clauses)

The `subst` helper function substitutes variables with given values in first-order logic statements.
This will be useful in later algorithms.
It's implementation is quite simple and self-explanatory.

In [176]:
print(inspect.getsource(subst))


def subst(s, x):
    """Substitute the substitution s into the expression x.
    >>> subst({x: 42, y:0}, F(x) + y)
    (F(42) + 0)
    """
    if isinstance(x, list):
        return [subst(s, xi) for xi in x]
    elif isinstance(x, tuple):
        return tuple([subst(s, xi) for xi in x])
    elif not isinstance(x, Expr):
        return x
    elif is_var_symbol(x.op):
        return s.get(x, x)
    else:
        return Expr(x.op, *[subst(s, arg) for arg in x.args])



Here's an example of how `subst` can be used.

In [177]:
subst({x: expr('Nono'), y: expr('M1')}, expr('Owns(x, y)'))

Owns(Nono, M1)

## Inference in First-Order Logic
In this section we look at a forward chaining and a backward chaining algorithm for `FolKB`. Both aforementioned algorithms rely on a process called <strong>unification</strong>, a key component of all first-order inference algorithms.

### Unification
We sometimes require finding substitutions that make different logical expressions look identical. This process, called unification, is done by the `unify` algorithm. It takes as input two sentences and returns a <em>unifier</em> for them if one exists. A unifier is a dictionary which stores the substitutions required to make the two sentences identical. It does so by recursively unifying the components of a sentence, where the unification of a variable symbol `var` with a constant symbol `Const` is the mapping `{var: Const}`. Let's look at a few examples.

In [178]:
unify(expr('x'), 3)

{x: 3}

In [179]:
unify(expr('A(x)'), expr('A(B)'))

{x: B}

In [180]:
unify(expr('Cat(x) & Dog(Dobby)'), expr('Cat(Bella) & Dog(y)'))

{x: Bella, y: Dobby}

In cases where there is no possible substitution that unifies the two sentences the function return `None`.

In [181]:
print(unify(expr('Cat(x)'), expr('Dog(Dobby)')))

None


We also need to take care we do not unintentionally use the same variable name. Unify treats them as a single variable which prevents it from taking multiple value.

In [182]:
print(unify(expr('Cat(x) & Dog(Dobby)'), expr('Cat(Bella) & Dog(x)')))

None


### Forward Chaining Algorithm
We consider the simple forward-chaining algorithm presented in <em>Figure 9.3</em>. We look at each rule in the knowledge base and see if the premises can be satisfied. This is done by finding a substitution which unifies each of the premise with a clause in the `KB`. If we are able to unify the premises, the conclusion (with the corresponding substitution) is added to the `KB`. This inferencing process is repeated until either the query can be answered or till no new sentences can be added. We test if the newly added clause unifies with the query in which case the substitution yielded by `unify` is an answer to the query. If we run out of sentences to infer, this means the query was a failure.

The function `fol_fc_ask` is a generator which yields all substitutions which validate the query.

In [183]:
print(inspect.getsource(fol_fc_ask))


def fol_fc_ask(kb, alpha):
    """
    [Figure 9.3]
    A simple forward-chaining algorithm.
    """
    # TODO: improve efficiency
    kb_consts = list({c for clause in kb.clauses for c in constant_symbols(clause)})

    def enum_subst(p):
        query_vars = list({v for clause in p for v in variables(clause)})
        for assignment_list in itertools.product(kb_consts, repeat=len(query_vars)):
            theta = {x: y for x, y in zip(query_vars, assignment_list)}
            yield theta

    # check if we can answer without new inferences
    for q in kb.clauses:
        phi = unify_mm(q, alpha)
        if phi is not None:
            yield phi

    while True:
        new = []
        for rule in kb.clauses:
            p, q = parse_definite_clause(rule)
            for theta in enum_subst(p):
                if set(subst(theta, p)).issubset(set(kb.clauses)):
                    q_ = subst(theta, q)
                    if all([unify_mm(x, q_) is None for x in kb.clauses + new]):
 

Let's find out all the hostile nations. Note that we only told the `KB` that Nono was an enemy of America, not that it was hostile.

In [184]:
answer = fol_fc_ask(crime_kb, expr('Hostile(x)'))
print(list(answer))

[{x: Nono}]


The generator returned a single substitution which says that Nono is a hostile nation. See how after adding another enemy nation the generator returns two substitutions.

In [185]:
crime_kb.tell(expr('Enemy(JaJa, America)'))
answer = fol_fc_ask(crime_kb, expr('Hostile(x)'))
print(list(answer))

[{x: Nono}, {x: JaJa}]


<strong><em>Note</em>:</strong> `fol_fc_ask` makes changes to the `KB` by adding sentences to it.

### Backward Chaining Algorithm
This algorithm works backward from the goal, chaining through rules to find known facts that support the proof. Suppose `goal` is the query we want to find the substitution for. We find rules of the form $\text{lhs} \implies \text{goal}$ in the `KB` and try to prove `lhs`. There may be multiple clauses in the `KB` which give multiple `lhs`. It is sufficient to prove only one of these. But to prove a `lhs` all the conjuncts in the `lhs` of the clause must be proved. This makes it similar to <em>And/Or</em> search.

#### OR
The <em>OR</em> part of the algorithm comes from our choice to select any clause of the form $\text{lhs} \implies \text{goal}$. Looking at all rules's `lhs` whose `rhs` unify with the `goal`, we yield a substitution which proves all the conjuncts in the `lhs`. We use `parse_definite_clause` to attain `lhs` and `rhs` from a clause of the form $\text{lhs} \implies \text{rhs}$. For atomic facts the `lhs` is an empty list.

In [186]:
print(inspect.getsource(fol_bc_or))


def fol_bc_or(kb, goal, theta):
    for rule in kb.fetch_rules_for_goal(goal):
        lhs, rhs = parse_definite_clause(standardize_variables(rule))
        for theta1 in fol_bc_and(kb, lhs, unify_mm(rhs, goal, theta)):
            yield theta1



#### AND
The <em>AND</em> corresponds to proving all the conjuncts in the `lhs`. We need to find a substitution which proves each <em>and</em> every clause in the list of conjuncts.

In [187]:
print(inspect.getsource(fol_bc_and))


def fol_bc_and(kb, goals, theta):
    if theta is None:
        pass
    elif not goals:
        yield theta
    else:
        first, rest = goals[0], goals[1:]
        for theta1 in fol_bc_or(kb, subst(theta, first), theta):
            for theta2 in fol_bc_and(kb, rest, theta1):
                yield theta2



Now the main function `fl_bc_ask` calls `fol_bc_or` with substitution initialized as empty. The `ask` method of `FolKB` uses `fol_bc_ask` and fetches the first substitution returned by the generator to answer query. Let's query the knowledge base we created from `clauses` to find hostile nations.

In [188]:
# Rebuild KB because running fol_fc_ask would add new facts to the KB
crime_kb = FolKB(clauses)

In [189]:
crime_kb.ask(expr('Hostile(x)'))

{v_194: Nono, x: Nono}

You may notice some new variables in the substitution. They are introduced to standardize the variable names to prevent naming problems as discussed in the [Unification section](#Unification)

## Appendix: The Implementation of `|'==>'|`

Consider the `Expr` formed by this syntax:

In [190]:
P |'==>'| ~Q

(P ==> ~Q)

What is the funny `|'==>'|` syntax? The trick is that "`|`" is just the regular Python or-operator, and so is exactly equivalent to this: 

In [191]:
(P | '==>') | ~Q

(P ==> ~Q)

In other words, there are two applications of or-operators. Here's the first one:

In [192]:
P | '==>'

PartialExpr('==>', P)

What is going on here is that the `__or__` method of `Expr` serves a dual purpose. If the right-hand-side is another `Expr` (or a number), then the result is an `Expr`, as in `(P | Q)`. But if the right-hand-side is a string, then the string is taken to be an operator, and we create a node in the abstract syntax tree corresponding to a partially-filled  `Expr`, one where we know the left-hand-side is `P` and the operator is `==>`, but we don't yet know the right-hand-side.

The `PartialExpr` class has an `__or__` method that says to create an `Expr` node with the right-hand-side filled in. Here we can see the combination of the `PartialExpr` with `Q` to create a complete `Expr`:

In [193]:
partial = PartialExpr('==>', P) 
partial | ~Q

(P ==> ~Q)

This  [trick](http://code.activestate.com/recipes/384122-infix-operators/) is due to [Ferdinand Jamitzky](http://code.activestate.com/recipes/users/98863/), with a modification by [C. G. Vedant](https://github.com/Chipe1),
who suggested using a string inside the or-bars.

## Appendix: The Implementation of `expr`

How does `expr` parse a string into an `Expr`? It turns out there are two tricks (besides the Jamitzky/Vedant trick):

1. We do a string substitution, replacing "`==>`" with "`|'==>'|`" (and likewise for other operators).
2. We `eval` the resulting string in an environment in which every identifier
is bound to a symbol with that identifier as the `op`.

In other words,

In [194]:
expr('~(P & Q)  ==>  (~P | ~Q)')

(~(P & Q) ==> (~P | ~Q))

is equivalent to doing:

In [195]:
P, Q = symbols('P, Q')
~(P & Q)  |'==>'|  (~P | ~Q)

(~(P & Q) ==> (~P | ~Q))

One thing to beware of: this puts `==>` at the same precedence level as `"|"`, which is not quite right. For example, we get this:

In [196]:
P & Q  |'==>'|  P | Q

(((P & Q) ==> P) | Q)

which is probably not what we meant; when in doubt, put in extra parens:

In [197]:
(P & Q)  |'==>'|  (P | Q)

((P & Q) ==> (P | Q))

## Examples

In [198]:
from notebook import Canvas_fol_bc_ask
canvas_bc_ask = Canvas_fol_bc_ask('canvas_bc_ask', crime_kb, expr('Criminal(x)'))